In [1]:
import sqlite3
import pandas as pd

In [40]:
db_name = "../pinecone/data/db.db"
table_name = "job_data"

conn = sqlite3.connect(db_name)

# Dump the SQL query into a DataFrame
df = pd.read_sql_query(f"""
    SELECT jobLink, jobTitle, jobCompany, minSalary, maxSalary, jobDetails, jobLocation, pullDate
    FROM {table_name}
""", conn)

# Close the connection
conn.close()

In [60]:
"""
DELETE a table
"""

db_name = "../pinecone/data/db.db"
table_name = "ai_desc_data_v4"

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
conn.commit()

conn.close()

In [38]:
db_name = "../pinecone/data/db.db"
table_name = "ai_desc_data"

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Dump the SQL query into a DataFrame
df = pd.read_sql_query(f"""
    SELECT *
    FROM {table_name}
""", conn)

conn.close()

In [41]:
df.iloc[8]

jobLink        https://www.indeed.com/rc/clk?jk=21f38ec02c730...
jobTitle                              Data Engineer (L5) - Games
jobCompany                                               Netflix
minSalary                                                170,000
maxSalary                                                720,000
jobDetails     Job Requisition ID     JR29190     Job Posting...
jobLocation                                        Los Gatos, CA
pullDate                                              2024-10-23
Name: 8, dtype: object

In [52]:
prompt = f"""
            You are a helpful assistant that extracts structured information from job postings. Below is a job posting from Indeed:

            {df.iloc[8]["jobDetails"]}

            Extract the following information in JSON format:
            - "job_description": A reworded and slightly summarized version of the full job description. Include all responsibilities, required skills, and key tasks. If the job description is not clearly labeled, infer it from any relevant sections, such as role expectations, day-to-day activities, or skill requirements. Avoid leaving this field blank unless absolutely no relevant information is present. Avoid exact wording.
            - "requirements": A rephrased list of job requirements, maintaining the meaning but avoiding exact wording.
            - "experience": A string with the minimum years of experience required for this job, formatted like "3 Years" or "1 Year".
            - "company_description": A summarized and reworded version of the company description, if available.

            Maintain accuracy while avoiding direct copying of phrases from the original text. The rephrasing should provide the same essential information in a fresh and unique way.


            Return the output in the following JSON format:
            {{
                "job_description": "...",
                "requirements": ["...", "..."],
                "experience": "...",
                "company_description": "..."
            }}

            Do not return anything else but the data in JSON format.
            """

In [45]:
import requests
import json
API_URL = "http://192.168.1.27:11434/api/chat"

In [53]:
payload = {
    "model":"llama3.2",
    "messages": [
        {"role":"user",
        "content":prompt}
    ]
}

response = requests.post(API_URL,json=payload,stream=True)

if response.status_code != 200:
    print(f"Error {response.status_code} : {response.text}")

full_message = ""
for line in response.iter_lines():
    if line:
        try:
            message_data = json.loads(line)
            if "message" in message_data and "content" in message_data["message"]:
                full_message += message_data["message"]["content"]

        except json.JSONDecodeError as e:
            print(f"Error Decoding JSON: {e}")



In [54]:
temp = json.loads(full_message)
temp

{'job_description': 'The Data Engineer will build robust data pipelines to output high-quality data at scale, collaborating with stakeholders to develop software and data solutions for analysis across the entire library of games. They will also work closely with the logging infrastructure team to establish key schemas and manage cross-functional engineering projects with autonomy. Responsibilities include increasing automation and scalability of complex data sets and building production data pipelines using Spark, Flink, or Hive/Hadoop.',
 'requirements': ['7+ years in software development with experience in data processing for analytics',
  'high-level autonomy in managing projects',
  'experience with schema design and data modeling',
  'programming proficiency in Python, Scala, or Java',
  'strong SQL skills',
  'excellent communication skills'],
 'experience': '7 Years',
 'company_description': 'Netflix is a leading entertainment service provider with over 250 million paid members 

In [28]:
df['requirements'][10]

'["Requirement understanding and gathering", "Design and develop data pipelines for new module", "Unit testing and help UAT team for any issues", "Upload all the deliverables in Git and help prod team to deploy the code in prod", "Design and implement robust data transformation pipelines using pyspark", "Develop and maintain a scalable data pipeline architecture that supports a wide variety of data sources and business use cases.", "Collaborate with data engineers and business stakeholders to define requirements for data transformations and models.", "Lead best practices for pipeline development including modularity testing version control and continuous integration (CI/CD).", "Optimize data pipeline models for performance and scalability in a cloud-based data warehouse environment (AWS S3 Redshift).", "Ensure data quality and governance through well-defined data transformation processes testing frameworks and documentation.", "Troubleshoot and resolve issues related to data pipelines 

In [23]:
job_num = 17
cur_details = df['jobDetails'].iloc[job_num]
cur_details

'Job Description:Resources will assist clients with rack, stack and installation of key data center equipment.They will help troubleshoot common issues, work through various tickets and act as a level 1 administrator for any data center issues.Do mechanical assembly/disassembly and install heavy GPU trays into systemsBuild up prototype open board systems on the benchtop and debugging issues reported by users.Triage and debug systems and get systems to pass base line diagnostics.Requirements:Strong data center experience with rack and stack,Minimum of 5 years on-prem large scale data center experience supporting the Data Center equipment and/or infrastructure (Power, Space, Cooling, Equipment)Strong cabling skills, cable management and dressingStrong NW troubleshooting experience, ideally with Supermicro Servers, Infiniband Fabric and Mellanox, Arista HardwareStrong attention to detail, independent and out of box thinkingSolid skills in UNIX (Ubuntu or RedHat) administration and knowled

In [24]:
import re

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = text.strip()  # Trim leading and trailing spaces
    return text

def extract_job_type(text):
    job_types = ["full[- ]?time", "part[- ]?time", "contract", "temporary", "internship", "freelance"]
    pattern = re.compile("|".join(job_types), re.IGNORECASE)
    match = pattern.search(text)
    return match.group(0).capitalize() if match else "Not specified"

def is_remote(text):
    remote_keywords = ["remote", "work from home", "telecommute", "virtual"]
    pattern = re.compile("|".join(remote_keywords), re.IGNORECASE)
    return bool(pattern.search(text))

def extract_max_years_experience(job_details):
    """
    Extracts the maximum years of experience from a job description.

    Parameters:
    job_details (str): The job details text blob.

    Returns:
    str: Maximum years of experience or 'Not specified' if not found.
    """
    # Define regex patterns to capture years of experience
    patterns = [
        r'(\d{1,2})\+?\s*(?:years|yrs)\s*of\s*experience',  # e.g., '5 years of experience', '3+ years'
        r'(\d{1,2})-(\d{1,2})\s*(?:years|yrs)',             # e.g., '3-5 years'
        r'at least\s*(\d{1,2})\s*(?:years|yrs)'             # e.g., 'at least 2 years'
    ]
    
    # List to collect all found years of experience
    years = []

    # Loop through patterns and find all matches
    for pattern in patterns:
        matches = re.findall(pattern, job_details, re.IGNORECASE)
        for match in matches:
            # If it's a tuple (from a range), take the max of the range
            if isinstance(match, tuple):
                years.extend(map(int, match))
            else:
                years.append(int(match))

    # Return the max years found or 'Not specified' if the list is empty
    return max(years) if years else 'Not specified'

clean_details = preprocess_text(cur_details)
job_type = extract_job_type(clean_details)
is_remote = is_remote(clean_details)
yrs_exp = extract_max_years_experience(clean_details)

print("Job Type:", job_type)
print("Is Remote:", is_remote)
print("Yrs Exp:", yrs_exp)

Job Type: Full-time
Is Remote: False
Yrs Exp: Not specified


In [25]:
from langchain_ollama import OllamaLLM
import json

def create_prompt(job_details):
    prompt = f"""
    You are a helpful assistant that extracts structured information from job postings. Below is a job posting from Indeed:

    {job_details}

    Extract the following information in JSON format:
    - "job_description": The full text of the job description, including all responsibilities and skills needed as stated.
    - "requirements": A list of job requirements exactly as listed in the posting.
    - "employment_type": A string with either full-time or contract if the job is full-time or contract.
    - "experience": A string of the range of amount of years required for this job, return None if job posting does not mention.
    - "remote": A boolean, 1 if the job is remote, 0 if the job is not remote.
    - "company_description": The full text of the company description (if available).

    Maintain the wording and details as much as possible as they are presented. You can summarize lightly.

    Return the output in the following JSON format:
    {{
        "job_description": "...",
        "requirements": ["...", "..."],
        "employment_type": "...",
        "experience": "...",
        "remote": "...",
        "company_description": "..."
    }}

    Do not return anything else but the data in JSON format.
    """
    return prompt

def ask_ollama(prompt):
    model = OllamaLLM(model="llama3.2")
    response = model.invoke(prompt)
    return response

def parse_llm_response(response):
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        return None

def extract_job_details(job_details):
    # Preprocess the job details
    job_details = preprocess_text(job_details)
    
    # Create the prompt for Ollama
    prompt = create_prompt(job_details)
    
    # Ask Ollama to extract the information
    response = ask_ollama(prompt)
    
    # Parse the response into a dictionary
    structured_data = parse_llm_response(response)
    
    return structured_data

extracted_sections = extract_job_details(clean_details)

In [26]:
print("Job Description:",extracted_sections.get("job_description",""))
print("*"*20)
print("requirements:",extracted_sections.get("requirements",""))
print("*"*20)
print("exmployment Type:",extracted_sections.get("employment_type",""))
print("*"*20)
print("Experience:",extracted_sections.get("experience",""))
print("*"*20)
print("remote:",extracted_sections.get("remote",""))
print("*"*20)
print("Company Description:",extracted_sections.get("company_description",""))

Job Description: Resources will assist clients with rack, stack and installation of key data center equipment.They will help troubleshoot common issues, work through various tickets and act as a level 1 administrator for any data center issues.Do mechanical assembly/disassembly and install heavy GPU trays into systemsBuild up prototype open board systems on the benchtop and debugging issues reported by users.Triage and debug systems and get systems to pass base line diagnostics.
********************
requirements: ['Strong data center experience with rack and stack', 'Minimum of 5 years on-prem large scale data center experience supporting the Data Center equipment and/or infrastructure (Power, Space, Cooling, Equipment)', 'Strong cabling skills, cable management and dressing', 'Strong NW troubleshooting experience, ideally with Supermicro Servers, Infiniband Fabric and Mellanox, Arista Hardware', 'Strong attention to detail, independent and out of box thinking', 'Solid skills in UNIX (